<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# この章では、BERTを使って穴埋めタスクを行う


# ライブラリたち
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM


# ちなみに、BertForMaskedLMは特殊トークン[MASK]に入るトークンを語彙の中から予測するクラス

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
# まずはトークナイザを準備

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)


# 次は穴埋めタスク用の事前学習済みモデルを準備
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
# GPUにのっける
bert_mlm = bert_mlm.cuda()


AttributeError: ignored

In [24]:
# 「今日は[MASK]へ行く。」　を穴埋めしてみよう

# ・・・とその前に、まずは文章をトークン化したものを見てみよう（[MASK]がちゃんとトークンと見做されている）
text = "今日は[MASK]へ行く。"
tokens = tokenizer.tokenize(text)
print(tokens)


# 手順1：トークン列をトークンIDで置き換える(符号化)
input_ids = tokenizer.encode(text, return_tensors = 'pt')
print(input_ids)

# そしてGPUへ送り込む
input_ids = input_ids.cuda()


['今日', 'は', '[MASK]', 'へ', '行く', '。']
tensor([[   2, 3246,    9,    4,  118, 3488,    8,    3]])


In [25]:
# 手順２：BERTに入力して分類スコアを得る
# １文しか入力してないので、input_ids以外の指定（トークンの最大数とか）が必要ない
with torch.no_grad():
  output = bert_mlm(input_ids = input_ids)
# print(output)

# outputの属性のうち「logits」が、語彙中の各単語に対する分類スコアである
# scoresは、三次元配列：各次元数（サイズ）は(バッチサイズ, 系列長, 語彙のサイズ)
# scores[i, j, k] = 入力された文章のi文目に対応するトークン列の、j番目のトークンに対して、トークンIDがkの語彙のスコア
scores = output.logits
print(scores)

tensor([[[ -5.8525,   5.0457,  -1.7965,  ...,  -4.8386,  -6.4219,  -7.8085],
         [ -4.0218,   7.2845,  -5.3993,  ...,  -6.0369,  -6.5811,  -2.1289],
         [ -5.8364,   5.3641,  -2.2106,  ...,  -4.3529,  -5.7284,  -4.3889],
         ...,
         [ -7.8698,   5.9753,  -4.3922,  ...,  -4.3223,  -6.0900, -11.4386],
         [ -5.4500,   6.5491,   0.0368,  ...,  -4.5615,  -5.1636,  -7.0161],
         [ -8.7510,   3.2686,  -1.6596,  ...,  -5.0593,  -7.0547, -10.7624]]],
       device='cuda:0')


In [26]:
# ちなみにBertForMaskedMLは
# 入力 -> BertModelに入力を入れた時の出力 -> それを線形変換 -> GELU関数（活性化関数） -> 線形変換 -> 最終出力

In [27]:
# 手順３：scoresから[MASK]に入るトークンを予測

# まず、入力された文章（or　文章集合）から、[MASK]（こいつのトークンIDは4）の位置（配列のインデックス）を求める
# input_ids[i].tolist().index(4) : i文目の中でID4に対応するインデックス
mask_position = input_ids[0].tolist().index(4)

# スコアが最も良いトークンのIDを取り出す
# argmax：配列で、一番大きい要素の「インデックス（順番）」を返す関数。括弧の中は初期値（省略可能）
id_best = scores[0, mask_position].argmax(-1).item()

# id_bestに対応するトークンを入手
token_best = tokenizer.convert_ids_to_tokens(id_best)

# 取り出したトークンに「##」がついていた場合(Chapter4参照)は、それを取り除く
token_best = token_best.replace("##", "")

# 元の入力文章の{MASK}を、token_bestで置き換える
final_text = text.replace("[MASK]", token_best)
print(final_text)

今日は東京へ行く。


In [35]:
# 最上位１位だけでなく、上位１０位を求めてみよう

# まずは、text, tokenizer, bert_mlm, num_topk(=上位k件)を入力として、上位num_topk件の穴埋め予測を出す関数を定義
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):

  # テキストを符号化
  input_ids = tokenizer.encode(text, return_tensors='pt')
  input_ids = input_ids.cuda()



  # bert_mlmに入力（計算結果を保存しないことで、リソースを節約）
  with torch.no_grad():
    output = bert_mlm(input_ids = input_ids)
  # 分類スコアを取得
  scores = output.logits




  # トークンID ４　に対応する、input_idsのインデックスを求める
  mask_position = input_ids[0].tolist().index(4)

  # scoresから上位num_topk件を取得
  # topk(n) は上位n件を取得してくれる
  scores_topk = scores[0, mask_position].topk(num_topk)

  # scores_topkのスコアを持つトークンのID列
  # indices はnumpy が提供する関数っぽい。
  # scores_topkのscores[0, masked_position]内でのindex (=token id)を取得
  ids_topk = scores_topk.indices

  # ids_topkを対応するトークンへ変換
  tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)

  





  # 以上で求めた上位トークンで文中の[MASK]を置き換える
  text_topk = []
  for token in tokens_topk:
    token = token.replace('##', '')
    text_topk.append(text.replace('[MASK]', token))

  return text_topk



In [40]:
# 上記の関数で、上位１０件の文章を出力してみよう

text_topk = predict_mask_topk (text, tokenizer, bert_mlm, 10)

# * : 配列を展開
#  sep : 区切り方指定
# option + ¥ でバックスラッシュを打てる
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。
